# Week 2: Preprocessing Text (Part 2)


In [26]:
#necessary library imports and setup introduced previously

import sys
import re
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
from itertools import zip_longest
import nltk
from nltk.corpus import reuters
from nltk.tokenize import word_tokenize
#nltk.download('punkt')

In [ ]:
##uncomment these lines below if working on colab
from google.colab import drive
drive.mount('/content/drive/')

SyntaxError: invalid syntax (ipython-input-2580162314.py, line 3)

## Overview
Remember, a raw text document is just a sequence of characters. There are a number of basic steps that are often performed when processing natural language text. In lab sessions this week we are covering some of the basic text pre-processing methods. In the previous notebook, you looked at
- <b> segmentation</b> - breaking down large units of text into smaller units such as documents and sentences.
- <b> tokenisation</b> - roughly speaking, this involves grouping characters into words;

In this notebook, you will be looking at:
- <b>case normalisation</b> - this involves converting all of the text into lower case;
- <b>stemming</b> - this involves removing a word's inflections to find the stem; and
- <b>punctuation and stop-word removal</b> - stop-words are common functions words that in some situations can be ignored.

Note that we do not always apply all of the above preprocessing methods; it depends on the application. One of the things that you will be learning about in this module, is when the application of each of these methods is, and is not, appropriate.

## Normalising text and removing unimportant tokens
In this next section we will consider several methods that pre-process (tokenised) text in ways that are sometimes helpful to 'downstream' processing.

### Number and case normalisation
Without any kind of normalisation, the tokens `"help"` and `"Help"` are two distinct types. In some contexts you may not want to distinguish them.

Another example, is that `"1998"` and `"1999"` count as distinct types. There are situations where there is no need to distinction between different numbers.

The following code performs case normalisation and replaces tokens that consist of digits by "NUM".
- Python provides a [number of functions](http://docs.python.org/library/stdtypes.html#string-methods), which you can call in order to analyse their content, or produce new strings from them.
- The code uses a [list comprehension](http://docs.python.org/tutorial/datastructures.html#list-comprehensions) to build a new list by looping through and filtering items.

In [27]:
tokens = ["The","cake","is","a","LIE"]      #a list of tokens, some of which contain uppercase letters
print([token.lower() for token in tokens])   #print newly created list of all lowercase tokens

numbers = ['in', 'the', 'year', '120', 'of', 'the', 'fourth', 'age', ',', 'after', '120', 'years', 'as', 'king', ',' , 'aragorn', 'died', 'at', 'the', 'age', 'of', '210']
print(["NUM" if token.isdigit() else token for token in numbers])  #replace all number tokens with "NUM" in a new list of tokens

['the', 'cake', 'is', 'a', 'lie']
['in', 'the', 'year', 'NUM', 'of', 'the', 'fourth', 'age', ',', 'after', 'NUM', 'years', 'as', 'king', ',', 'aragorn', 'died', 'at', 'the', 'age', 'of', 'NUM']


### Exercise 1.1
- Write a function <code>number_normalise</code> which
    * replaces numbers with NUM;
    * and replaces tokens such as `"4th"`, `"1st"` and `"22nd"` with `"Nth"`.
- Test your code on the list `["Within","5","minutes",",","the", "1st", "and", "2nd", "placed", "runners", "lapped", "the", "5th","."]`.
- Check that the token `"and"` isn't changed to `"Nth"`.
- You will find [this page](http://docs.python.org/library/stdtypes.html#string-methods) useful.


In [28]:
def number_normalise(text):
  normalise_token = []
  for token in text:
    if token.isdigit():
      normalise_token.append("NUM")

    elif token[-2:] in ["st", "nd", "rd", "th"] and token[:-2].isdigit():
      normalise_token.append("Nth")
    else:
      normalise_token.append(token)

  return normalise_token


In [29]:
text = ["Within","5","minutes",",","the", "1st", "and", "2nd", "placed", "runners", "lapped", "the", "5th","."]
number_normalise(text)

['Within',
 'NUM',
 'minutes',
 ',',
 'the',
 'Nth',
 'and',
 'Nth',
 'placed',
 'runners',
 'lapped',
 'the',
 'Nth',
 '.']

### Exercise 1.2
- Complete the code in the cell below. You have just two lines to complete. The goal is to use a large sample of the Reuters corpus to establish the extent to which vocabulary size is reduced when number and case normalisation is applied.
- For each of the two incomplete lines you should ideally use nested list comprehensions. This is described in Section 5.1.4 in [this document](http://docs.python.org/tutorial/datastructures.html#list-comprehensions).  Alternatively, you could define functions which iterate over the sentences in each sample and the tokens within each sentence.


In [30]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [31]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [32]:
#the sample_sentences() function from the last lab will be useful here
#next week we will look at including useful functions like this in a utils python file which we can import from
import random

def sample_sentences(corpus,sample_size):

    size=len(corpus)
    ids=random.sample(range(size),sample_size)
    sample=[corpus[i] for i in ids]
    return sample

#in vocabulary_size(), we use a dictionary to store the frequency of each token type in the corpus
#the number of keys in this dictionary is the size of the vocab
def vocabulary_size(sentences):
    tok_counts = {}
    for sentence in sentences:
        for token in sentence:
            tok_counts[token]=tok_counts.get(token,0)+1
    return len(tok_counts.keys())



sample_size = 10000

raw_sentences = sample_sentences(reuters.sents(),sample_size)

############################################
lowered_sentences = [[token.lower() for token in sentence] for sentence in raw_sentences]
normalised_sentences = [number_normalise(sentence) for sentence in lowered_sentences]

############################################

raw_vocab_size = vocabulary_size(raw_sentences)
normalised_vocab_size = vocabulary_size(normalised_sentences)
print("Normalisation produced a {0:.2f}% reduction in vocabulary size from {1} to {2}".format(
    100*(raw_vocab_size - normalised_vocab_size)/raw_vocab_size,raw_vocab_size,normalised_vocab_size))


Normalisation produced a 28.19% reduction in vocabulary size from 20139 to 14461


## Stemming
A considerable amount of the lexical variation found in documents results from the use of morphological variants which we might not wish to distinguish - e.g. when determining the topic of a document. An easy way to remove these varied forms is to use a stemmer. NLTK includes a number of stemmers in the `nltk.stem` package.
- [NLTK stem module API](https://www.nltk.org/api/nltk.stem.html)

- [NLTK Porter stemmer](https://www.nltk.org/api/nltk.stem.porter.html)

- Look at the code below to show how the NLTK implementation of the Porter stemmer in `nltk.stem.porter.PorterStemmer` stems a sample of sentences in the Reuters corpus.
- Have a close look at the differences between the columns. This will give you a good indication of what the stemmer does.

In [33]:
from nltk.stem.porter import PorterStemmer

st = PorterStemmer()

sample_size = 10

tokenised_sentences = sample_sentences(reuters.sents(),sample_size)

for sentence in tokenised_sentences:
    df = pd.DataFrame(list(zip_longest(sentence,[st.stem(token) for token in sentence])),columns=["BEFORE","AFTER"])
    display(df)

,BEFORE,AFTER
0,Foreign,foreign
1,business,busi
2,and,and
3,securities,secur
4,trading,trade
5,turnover,turnov
6,also,also
7,increased,increas
8,.,.


,BEFORE,AFTER
0,1985,1985
1,4th,4th
2,qtr,qtr
3,and,and
4,year,year
5,oper,oper
6,net,net
7,excludes,exclud
8,a,a
9,gain,gain


,BEFORE,AFTER
0,The,the
1,company,compani
2,said,said
3,the,the
4,alternative,altern
5,filing,file
6,seeks,seek
7,to,to
8,recover,recov
9,about,about


,BEFORE,AFTER
0,The,the
1,NAPM,napm
2,said,said
3,the,the
4,economic,econom
5,improvement,improv
6,was,wa
7,evident,evid
8,in,in
9,all,all


,BEFORE,AFTER
0,Hungary,hungari
1,is,is
2,expanding,expand
3,its,it
4,beet,beet
5,area,area
6,to,to
7,105,105
8,",",","
9,000,000


,BEFORE,AFTER
0,The,the
1,yield,yield
2,of,of
3,the,the
4,key,key
5,5,5
6,.,.
7,1,1
8,pct,pct
9,89th,89th


,BEFORE,AFTER
0,The,the
1,Bundesbank,bundesbank
2,has,ha
3,also,also
4,ruled,rule
5,out,out
6,a,a
7,quick,quick
8,cut,cut
9,in,in


,BEFORE,AFTER
0,The,the
1,sale,sale
2,of,of
3,CGCT,cgct
4,",",","
5,which,which
6,controls,control
7,16,16
8,pct,pct
9,of,of


,BEFORE,AFTER
0,Papandreou,papandr
1,has,ha
2,maintained,maintain
3,in,in
4,the,the
5,past,past
6,that,that
7,he,he
8,will,will
9,not,not


,BEFORE,AFTER
0,The,the
1,yield,yield
2,on,on
3,the,the
4,bills,bill
5,for,for
6,sale,sale
7,to,to
8,banks,bank
9,and,and


### Exercise 2.1
- By looking at the impact on a large sample of the Reuters corpus, establish the extent to which vocabulary size is reduced by stemming.
- Write code to do this in the empty cell below. You should be able to re-use a lot of the code from the code you used when measuring the impact of lower case and number normalisation.

In [39]:
import random

def sample_sentences(corpus,sample_size):

    size=len(corpus)
    ids=random.sample(range(size),sample_size)
    sample=[corpus[i] for i in ids]
    return sample

#in vocabulary_size(), we use a dictionary to store the frequency of each token type in the corpus
#the number of keys in this dictionary is the size of the vocab
def vocabulary_size(sentences):
    tok_counts = {}
    for sentence in sentences:
        for token in sentence:
            tok_counts[token]=tok_counts.get(token,0)+1
    return len(tok_counts.keys())



sample_size = 10000

raw_sentences = sample_sentences(reuters.sents(),sample_size)

############################################
normalised_sentences=[number_normalise(sentence) for sentence in raw_sentences]
stem_sentences = [[st.stem(token) for token in sentence] for sentence in normalised_sentences]


############################################

raw_vocab_size = vocabulary_size(raw_sentences)
stem_vocab_size = vocabulary_size(stem_sentences)
print("Stemming produced a {0:.2f}% reduction in vocabulary size from {1} to {2}".format(
    100*(raw_vocab_size - stem_vocab_size)/raw_vocab_size,raw_vocab_size,stem_vocab_size))

Stemming produced a 47.31% reduction in vocabulary size from 20176 to 10631


### Exercise 2.2
* Try using the WordNetLemmatizer <code>nltk.stem.wordnet.WordNetLemmatizer</code> instead of the Porter Stemmer.
* Using a large sample of the Reuters corpus, establish the extent to which the vocabulary size reduced by lemmatization?
* As an extension, you could look at different sample sizes and/or different corpora and display the results in a table or graph (e.g., using <code>pandas</code> and/or <code>matplotlib</code>)

In [36]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [40]:

from nltk.stem.wordnet import WordNetLemmatizer
import random

wnl = WordNetLemmatizer()

def sample_sentences(corpus,sample_size):

    size=len(corpus)
    ids=random.sample(range(size),sample_size)
    sample=[corpus[i] for i in ids]
    return sample

#in vocabulary_size(), we use a dictionary to store the frequency of each token type in the corpus
#the number of keys in this dictionary is the size of the vocab
def vocabulary_size(sentences):
    tok_counts = {}
    for sentence in sentences:
        for token in sentence:
            tok_counts[token]=tok_counts.get(token,0)+1
    return len(tok_counts.keys())



sample_size = 10000

raw_sentences = sample_sentences(reuters.sents(),sample_size)

############################################
normalised_sentences=[number_normalise(sentence) for sentence in raw_sentences]
lemma_sentences = [[wnl.lemmatize(token) for token in sentence] for sentence in normalised_sentences]


############################################

raw_vocab_size = vocabulary_size(raw_sentences)
lemma_vocab_size = vocabulary_size(lemma_sentences)
print("Normalisation produced a {0:.2f}% reduction in vocabulary size from {1} to {2}".format(
    100*(raw_vocab_size - lemma_vocab_size)/raw_vocab_size,raw_vocab_size,lemma_vocab_size))


Normalisation produced a 11.34% reduction in vocabulary size from 20108 to 17827


### Punctuation and stop-word removal
A stopword is a word that occurs so often that it loses its usefulness in some tasks. We may get more meaningful information from our corpus analysis if we remove stopwords and punctuation.

The code below takes a list of tokens and creates a new list, which contains only those strings which are alphabetic and non-stop-words.

In [41]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:

stop = stopwords.words('english')
tokens="The cat , which is really fat , sat on the mat".lower().split()
filtered_tokens = [w for w in tokens if w.isalpha() and w not in stop]
print(tokens)
print(filtered_tokens)

['the', 'cat', ',', 'which', 'is', 'really', 'fat', ',', 'sat', 'on', 'the', 'mat']
['cat', 'really', 'fat', 'sat', 'mat']


**Note**: `isalpha` only returns `True` if the string is entirely composed of alphabet characters. If you want a function to return `True` even when a word contains digits, then you should use `isalnum`.`

### Exercise 3.1
- In the empty cell below, write code that looks at a large sample of the Reuters corpus, establishing what proportion of tokens are stop-words.
- As extension, you could establish the mean (and or the distribution of the) number of stop-words per sentence; or compare the numbers of stop-words in different corpora.

In [44]:
sample_size = 10000

raw_sentences = sample_sentences(reuters.sents(),sample_size)

normalised_sentences=[number_normalise(sentence) for sentence in raw_sentences]
lemma_sentences = [[wnl.lemmatize(token) for token in sentence] for sentence in normalised_sentences]

stop_word_exc = [[w for w in sentence if w.isalpha() and w not in stop] for sentence in lemma_sentences]
stop_word_exc

[['The',
  'company',
  'said',
  'could',
  'bring',
  'earnings',
  'NUM',
  'Columbia',
  'stated',
  'goal',
  'earning',
  'less',
  'NUM',
  'NUM',
  'dlrs',
  'per',
  'share',
  'dividend'],
 ['It',
  'also',
  'said',
  'Michael',
  'Gruenberg',
  'resigned',
  'company',
  'board',
  'director'],
 ['Union',
  'Texas',
  'said',
  'planned',
  'spend',
  'NUM',
  'mln',
  'dlrs',
  'next',
  'two',
  'year',
  'develop',
  'Eugene',
  'Island',
  'Block',
  'NUM',
  'Gulf',
  'Mexico'],
 ['Yu',
  'said',
  'government',
  'could',
  'maintain',
  'framework',
  'foreign',
  'exchange',
  'control',
  'finding',
  'way',
  'ease'],
 ['WICHITA',
  'lt',
  'WRO',
  'TO',
  'BUY',
  'FOUNTAIN',
  'OIL',
  'lt',
  'FGAS',
  'Wichita',
  'Industries',
  'Inc',
  'said',
  'agreed',
  'buy',
  'Fountain',
  'Oil',
  'Gas',
  'Inc',
  'Wichita',
  'said',
  'acquire',
  'outstanding',
  'share',
  'Fountain',
  'exchange',
  'NUM',
  'mln',
  'newly',
  'issued',
  'Wichita',
  'commo

In [49]:
number_token = 0
number_stop = 0

for sentence in lemma_sentences:
  for token in sentence:
    number_token +=1
    if token in stop:
        number_stop += 1

print(number_token)
print(number_stop)

313878
17345


In [ ]:
### Exercise 3.2
Explain the difference between the number of tokens in a corpus and the size of the vocabulary of a corpus.
Would you expect stopword removal to have a greater effect on the size of the vocabulary or the number of tokens in the corpus?

In [ ]:
#### My answer
